In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
def createObjective(input_df: pd.DataFrame) -> float:
    def objective(trial):
        # Эти параметры мы оптимзируем
        params = {
            'buy_th': trial.suggest_float("buy_th", 0.1, 0.1),
            'sell_th': trial.suggest_float("sell_th", -0.1, -0.1),
            'RSI_signal': trial.suggest_float("RSI_signal", -1.0, 1.0),
            'EMA_Cross_signal': trial.suggest_float("EMA_Cross_signal", -1.0, 1.0),
            'macd_signal':trial.suggest_float("macd_signal", -1.0, 1.0),
            'sma_signal':trial.suggest_float("sma_signal", -1.0, 1.0),
            'bb_signal': trial.suggest_float("bb_signal", -1.0, 1.0),
            'OBV_Signal':trial.suggest_float("OBV_Signal", -1.0, 1.0),
            'AD_Signal': trial.suggest_float("AD_Signal", -1.0, 1.0),
            'MFI_Signal':trial.suggest_float("MFI_Signal", -1.0, 1.0),
            'VO_Signal':trial.suggest_float("VO_Signal", -1.0, 1.0),
            'Stochastic_Signal': trial.suggest_float("Stochastic_Signal", -1.0, 1.0),
            'CCI_Signal': trial.suggest_float("CCI_Signal", -1.0, 1.0)
        }

        input_df['Open'] = input_df['open']
        input_df['High'] = input_df['high']
        input_df['Low'] = input_df['low']
        input_df['Close'] = input_df['close']
        input_df['Volume'] = input_df['volume']
    
        input_df['combined_signal'] = input_df['RSI_signal'] * params['RSI_signal']
        + input_df['EMA_Cross_signal'] * params['EMA_Cross_signal']
        + input_df['macd_signal'] * params['macd_signal']
        + input_df['sma_signal'] * params['sma_signal']
        + input_df['bb_signal'] * params['bb_signal']
        + input_df['OBV_Signal'] * params['OBV_Signal']
        + input_df['AD_Signal'] * params['AD_Signal']
        + input_df['MFI_Signal'] * params['MFI_Signal']
        + input_df['VO_Signal'] * params['VO_Signal']
        + input_df['Stochastic_Signal'] * params['Stochastic_Signal']
        + input_df['CCI_Signal'] * params['CCI_Signal']
        scaler = MinMaxScaler()
        input_df['combined_signal'] = scaler.fit_transform(input_df[['combined_signal']])
        input_df['Signal'] = 0
        input_df.loc[input_df['combined_signal'] > params['buy_th'], 'Signal'] = 1
        input_df.loc[input_df['combined_signal'] < params['sell_th'], 'Signal'] = -1
        bt = Backtest(input_df, SimpleFollowSignalsStrategy, cash=2*input_df.iloc[0]['close'], commission=.002, exclusive_orders=True)
        stats = bt.run()
        return stats['Return [%]']
    
    return objective

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(createObjective(input_df=signals_df), n_trials=500)
print(study.best_trial.params)

In [ ]:
def bulkTest(test_df: pd.DataFrame, signals: list) -> list:
    results = {}
    for signal_code in signals:
        test_df = signals_df.copy()
        test_df['Signal'] = 0
        test_df.loc[test_df[signal_code] == 1, 'Signal'] = 1
        test_df.loc[test_df[signal_code] == -1, 'Signal'] = -1
        test_df['Open'] = test_df['open']
        test_df['Close'] = test_df['close']
        test_df['High'] = test_df['high']
        test_df['Low'] = test_df['low']
        test_df['volume'] = test_df['volume']
        bt = Backtest(test_df, SimpleFollowSignalsStrategy, cash=2*test_df.iloc[0]['close'], commission=.002, exclusive_orders=True)
        stats = bt.run()
        results[signal_code] = stats['Return [%]']

    return results



In [ ]:
bulkTest(signals_df, ['RSI_signal', 'EMA_Cross_signal', 'macd_signal', 'sma_signal', 'bb_signal', 'OBV_Signal', 'AD_Signal', 'MFI_Signal', 'VO_Signal','Stochastic_Signal', 'CCI_Signal','train_signal_command','Signal'])

In [ ]:
y_test = y_test.to(cpu)
y_train = y_train.to(cpu)
# predicted_actions = [action_classes[i] for i in predicted_labels]
test_actions = [action_classes[i] for i in torch.argmax(y_test, dim=1).numpy()]
train_actions = [action_classes[i] for i in torch.argmax(y_train, dim=1).numpy()]
predicted_labels = [action_classes[i] for i in predicted_args]

# Визуализация предсказаний
plt.figure(figsize=(12, 6))
# plt.plot(range(len(y_train)), train_actions, label='Train True Actions', color='cyan')
plt.plot(range(0, len(y_test)), test_actions, label='True Actions', color='blue')
plt.plot(range(0,   len(y_test)), predicted_labels, label='Predicted Actions', color='red')
plt.legend()
plt.show()

torch.save(model.state_dict(), './models/action_forcast_lstm_model.pth')